<a href="https://colab.research.google.com/github/satvik314/RAG_experiments/blob/main/RAG_Gemini_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet langchain langchain-google-genai unstructured pypdf

In [ ]:
import os
from getpass import getpass

os.environ['GOOGLE_API_KEY'] = getpass()

··········


In [ ]:
from langchain import PromptTemplate
from langchain.document_loaders import WebBaseLoader, UnstructuredPDFLoader, PyPDFLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document

In [ ]:
#loading the data from the website
loader = WebBaseLoader("https://buildfastwithai.com")
docs = loader.load()

print(docs)

[Document(page_content='Build Fast with AI', metadata={'source': 'https://buildfastwithai.com', 'title': 'Build Fast with AI', 'description': 'Build Fast with AI - a vibrant community of AI builders, innovators, and enthusiasts. Whether you are an entrepreneur, a product manager, a developer, or anyone intrigued by AI, this is your platform to learn, grow, and innovate.', 'language': 'en'})]


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# load the model
llm = ChatGoogleGenerativeAI(model = "gemini-pro",
                             temperature = 0.7,
                             top_p = 0.85
                             )

## Summarise the document

In [ ]:
# creating a prompt template

doc_prompt = PromptTemplate.from_template("{page_content}")

llm_prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:
"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

print(llm_prompt)

input_variables=['text'] template='Write a concise summaru of the following:\n"{text}"\nCONCISE SUMMARY: \n'


In [ ]:
# Create stuff document chain using LCEL

stuff_chain = (
    # Extract data from the documents and add to the key `text`.
    {
        "text": lambda docs: "\n\n".join(
            format_document(doc, doc_prompt) for doc in docs
        )
    }
    | llm_prompt         # Prompt for Gemini
    | llm                # Gemini function
    | StrOutputParser()  # output parser
)


In [ ]:
stuff_chain.invoke(docs)

'- AI-powered tools can accelerate software development by automating repetitive tasks, generating code, and improving code quality.\n- AI can assist in building faster and more reliable applications by identifying potential issues early, optimizing performance, and enhancing security.\n- AI-driven insights can help developers make informed decisions, leading to improved productivity and efficiency.\n- By leveraging AI, developers can focus on higher-level tasks, fostering innovation and creativity.\n- AI-powered tools can enhance collaboration and communication among team members, streamlining the development process.'

In [ ]:
#loading the data from the PDF
loader = PyPDFLoader("/content/GenAI_CrashCourse_Info.pdf")
pdf_docs = loader.load()

print(pdf_docs)

[Document(page_content='Crash\nCourse\n-\nGenerative\nAI\nTransform\nYour\nIdeas\ninto\nReal-W orld\nApplications\nwith\nGenerative\nAI\nIntroduction\n"Step\ninto\nthe\nfuturistic\nworld\nof\nGenerative\nAI\nwith\nour\ncomprehensive\n4-week\ncourse.\nThis\nprogram\nis\nmeticulously\ncrafted\nto\nprovide\nyou\nwith\na\nﬁrm\ngrounding\nin\nPython\nprogramming\nand\nGenerative\nAI\ntechnologies,\nempowering\nyou\nto\nbuild\ninnovative\napplications\nthat\ncan\ntransform\nindustries.\nWith\na\nblend\nof\ntheoretical\nknowledge\nand\nhands-on\nexperience,\nembark\non\na\njourney\nto\nbecoming\na\nproﬁcient\nAI\ndeveloper."\nWho\ncan\ntake\nthis\ncourse?\nThis\ncourse\nis\nmeticulously\ndesigned\nfor:\n●\nAspiring\nAI\nDevelopers\nlooking\nto\nhone\ntheir\nskills\nand\nbuild\ngroundbreaking\napplications.\n●\nProfessionals\naiming\nto\nleverage\nAI\ntechnology\nto\nenhance\ntheir\nindustry\nexpertise.\n●\nStudents\nand\nHobbyists\nseeking\nto\ndelve\ndeep\ninto\nthe\nfascinating\nworld\nof\n

In [ ]:
stuff_chain.invoke(pdf_docs)

'- This 4-week course provides a comprehensive introduction to Generative AI and its applications in real-world scenarios.\n- It is designed for aspiring AI developers, professionals, students, and hobbyists with a basic understanding of Python programming.\n- The course includes project-based learning, hands-on experience, and a curriculum aligned with industry demands.\n- Participants will build two real-world projects and receive a certificate upon successful completion.\n- The course fee is Rs 20,000 (with a 50% discount for the first cohort), and registration is available through the website.\n- The course is led by Satvik, an experienced data scientist and Generative AI consultant, who emphasizes practical application and learning by doing.\n- Frequently asked questions address course fees, target audience, projects, learning outcomes, certification, missed classes, and contact information for further queries.'

## Query the Document

In [ ]:
query_prompt_template = """

<<
"{text}"
>>

Use the above information as a context to the answer the following query:
"{query}"
"""

query_prompt = PromptTemplate.from_template(query_prompt_template)

print(query_prompt)

input_variables=['query', 'text'] template='\n\n<<\n"{text}"\n>>\n\nUse the above information as a context to the answer the following query:\n"{query}"\n'


In [ ]:
query_stuff_chain = (
      # Extract data from the documents and add to the key `text`.
      {
          "text": lambda docs: "\n\n".join(
              format_document(doc, doc_prompt) for doc in docs
          ),
          "query" : "How can I benefit from this course?"
      }
      | llm_prompt         # Prompt for Gemini
      | llm                # Gemini function
      | StrOutputParser()  # output parser
  )

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

In [ ]:
# Create stuff document chain using LCEL

def query_doc(query):
  query_stuff_chain = (
      # Extract data from the documents and add to the key `text`.
      {
          "text": lambda docs: "\n\n".join(
              format_document(doc, doc_prompt) for doc in docs
          ),
          "query" : query
      }
      | llm_prompt         # Prompt for Gemini
      | llm                # Gemini function
      | StrOutputParser()  # output parser
  )

  query_stuff_chain.invoke()
